In [11]:

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate,ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, model="gpt-4-turbo-preview", streaming=True,callbacks=[StreamingStdOutCallbackHandler()])



ValidationError: 2 validation errors for ChatOpenAI
callbacks -> 0
  instance of BaseCallbackHandler expected (type=type_error.arbitrary_type; expected_arbitrary_type=BaseCallbackHandler)
callbacks
  instance of BaseCallbackManager expected (type=type_error.arbitrary_type; expected_arbitrary_type=BaseCallbackManager)

In [4]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
  def parse(self,text):
    items=  text.strip().split(',')
    return list(map(str.strip,items))
  
  

In [5]:
template = ChatPromptTemplate.from_messages([
  ("system","You are a list generating machine. Everyhitng you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do NOT reply with anything else."),
  ("human", "{question}")
])



In [8]:
# chain = template | chat | CommaOutputParser()

# chain.invoke({
#   "max_items" : 5,
#   "question": "what are the pokemons?"
# })
chef_prompt = ChatPromptTemplate.from_messages([
  ("system","You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients"),
  ("human","I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat 

In [10]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)


veg_chain = veg_chef_prompt | chat


final_chain = {"recipe" :chef_chain} | veg_chain

final_chain.invoke({"cuisine":"indian"})


AIMessage(content="To adapt the Chicken Tikka Masala recipe to a vegetarian version, we can replace the chicken with a plant-based alternative that still absorbs the rich flavors of the marinade and sauce. Paneer (Indian cottage cheese) or extra-firm tofu are excellent choices for this. Both alternatives offer a texture that can closely mimic chicken when cooked properly and will soak up the marinade and sauce, providing a satisfying vegetarian dish. Here's how you can modify the recipe:\n\n### Vegetarian Tikka Masala\n\n#### Ingredients:\n\n**For the Paneer or Tofu Marinade:**\n- 1 pound (about 450g) paneer or extra-firm tofu, cut into bite-sized pieces\n- 1 cup plain yogurt (use dairy-free yogurt for a vegan version)\n- 1 tablespoon lemon juice\n- 2 teaspoons ground cumin\n- 1 teaspoon ground cinnamon\n- 2 teaspoons cayenne pepper (adjust according to your spice preference)\n- 2 teaspoons freshly ground black pepper\n- 1 tablespoon minced fresh ginger\n- 1 teaspoon salt\n\n**For the 